In [1]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql as sql
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [2]:
conf = SparkConf().setAppName("report_analysis").setMaster('yarn')

In [3]:
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [4]:
posts_sample = spark.read.format('xml')\
.option('rootTag', 'posts')\
.option('rowTag', 'row')\
.load("posts_sample.xml")

In [5]:
posts_sample.printSchema()

root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



In [6]:
posts_sample.show()

+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+--------------------+--------------------+----------+
|_AcceptedAnswerId|_AnswerCount|               _Body|_ClosedDate|_CommentCount| _CommunityOwnedDate|       _CreationDate|_FavoriteCount|    _Id|   _LastActivityDate|       _LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|               _Tags|              _Title|_ViewCount|
+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+--------------------+----------

In [64]:
posts_sample_frm = posts_sample.select("_CreationDate", "_ViewCount", "_Tags")
posts_sample_frm.show()

+--------------------+----------+--------------------+
|       _CreationDate|_ViewCount|               _Tags|
+--------------------+----------+--------------------+
|2008-08-01 02:42:...|     42817|<c#><floating-poi...|
|2008-08-01 03:08:...|     18214|<html><css><inter...|
|2008-08-01 03:17:...|      null|                null|
|2008-08-01 04:40:...|    555183|<c#><.net><datetime>|
|2008-08-01 04:55:...|    149445|<c#><datetime><ti...|
|2008-08-01 04:56:...|      null|                null|
|2008-08-01 05:42:...|    176405|<html><browser><t...|
|2008-08-01 05:59:...|    123231|        <.net><math>|
|2010-09-20 20:18:...|      null|                null|
|2010-09-20 22:36:...|      null|                null|
|2010-09-20 23:04:...|      null|                null|
|2010-09-21 06:07:...|      null|                null|
|2010-09-21 11:33:...|      null|                null|
|2010-09-21 15:46:...|      null|                null|
|2010-09-21 16:07:...|      null|                null|
|2010-09-2

In [65]:
parsed_posts_sample = posts_sample_frm.filter("_Tags is not null").rdd.map(lambda x: (str(x[0]).split('-')[0], x[1], x[2][1:-1].split('><'))).toDF(["year", "views", "languages"])
parsed_posts_sample = parsed_posts_sample.filter(col("year") >= 2010).filter(col("year") <= 2020)
parsed_posts_sample.show(truncate=False)

+----+-----+---------------------------------------------------------------+
|year|views|languages                                                      |
+----+-----+---------------------------------------------------------------+
|2010|3650 |[c++, character-encoding]                                      |
|2010|617  |[sharepoint, infopath]                                         |
|2010|1315 |[iphone, app-store, in-app-purchase]                           |
|2010|973  |[symfony1, schema, doctrine, fixtures]                         |
|2010|132  |[java]                                                         |
|2010|419  |[visual-studio-2010, stylecop]                                 |
|2010|869  |[cakephp, file-upload, swfupload]                              |
|2010|1303 |[git, cygwin, putty]                                           |
|2010|748  |[drupal, drupal-6]                                             |
|2010|1258 |[php, wordpress, memory]                                       |

In [66]:
from pyspark.sql.functions import explode

parsed_posts_sample = parsed_posts_sample.withColumn("language", explode(parsed_posts_sample["languages"])).drop('languages')
parsed_posts_sample.show()

+----+-----+------------------+
|year|views|          language|
+----+-----+------------------+
|2010| 3650|               c++|
|2010| 3650|character-encoding|
|2010|  617|        sharepoint|
|2010|  617|          infopath|
|2010| 1315|            iphone|
|2010| 1315|         app-store|
|2010| 1315|   in-app-purchase|
|2010|  973|          symfony1|
|2010|  973|            schema|
|2010|  973|          doctrine|
|2010|  973|          fixtures|
|2010|  132|              java|
|2010|  419|visual-studio-2010|
|2010|  419|          stylecop|
|2010|  869|           cakephp|
|2010|  869|       file-upload|
|2010|  869|         swfupload|
|2010| 1303|               git|
|2010| 1303|            cygwin|
|2010| 1303|             putty|
+----+-----+------------------+
only showing top 20 rows



In [67]:
wiki_languages = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.csv("programming-languages.csv")

In [68]:
names = wiki_languages.select("name").rdd.map(lambda x: x[0].lower()).collect()
parsed_posts_sample = parsed_posts_sample.filter(parsed_posts_sample.language.isin(names))
parsed_posts_sample.show()

+----+-----+-----------+
|year|views|   language|
+----+-----+-----------+
|2010|  132|       java|
|2010| 1258|        php|
|2010| 9649|       ruby|
|2010| 2384|          c|
|2010| 1987|        php|
|2010| 3321|     python|
|2010|  128| javascript|
|2010|  477|applescript|
|2010| 1748|        php|
|2010|  998|        php|
|2010| 2095| javascript|
|2010|  447|        sed|
|2010| 6558|     python|
|2010|  214|       java|
|2010|  214|       ruby|
|2010|  852|objective-c|
|2010|  179| javascript|
|2010| 6709|          r|
|2010|   78|        php|
|2010| 1280| javascript|
+----+-----+-----------+
only showing top 20 rows



In [69]:
parsed_posts_sample = parsed_posts_sample.groupBy("year", "language").agg({"views": "sum"})
parsed_posts_sample.show()

+----+-----------+----------+
|year|   language|sum(views)|
+----+-----------+----------+
|2013|     erlang|      2302|
|2017| typescript|     29031|
|2017|        sed|        93|
|2013| javascript|    609571|
|2013|         f#|      4317|
|2012| powershell|     17311|
|2019|        php|      3753|
|2017|    haskell|      4040|
|2013| autohotkey|      4421|
|2013|applescript|      2059|
|2019|      xpath|        27|
|2015|     racket|       762|
|2017|         go|      1356|
|2018|      perl6|       109|
|2015|       dart|       402|
|2015|       rust|       117|
|2012|         f#|      1222|
|2018|     python|     99996|
|2017|     prolog|        76|
|2016|      latex|       461|
+----+-----------+----------+
only showing top 20 rows



In [70]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number

window = Window.partitionBy(parsed_posts_sample['year']).orderBy(parsed_posts_sample['sum(views)'].desc())

parsed_posts_sample_top10 = parsed_posts_sample.select('*', row_number().over(window).alias('row_number')).filter(col('row_number') <= 10)

In [75]:
parsed_posts_sample_top10 = parsed_posts_sample_top10.orderBy(col("year").asc(), col("sum(views)").desc()).drop("rank")
parsed_posts_sample_top10 = parsed_posts_sample_top10.withColumnRenamed("sum(views)", "views")
parsed_posts_sample_top10.show()

+----+-----------+-------+
|year|   language|  views|
+----+-----------+-------+
|2010|        php|1189629|
|2010|       java| 563211|
|2010| javascript| 316131|
|2010|objective-c|  97009|
|2010|       ruby|  76215|
|2010|          c|  66587|
|2010|     python|  60672|
|2010|     matlab|  51865|
|2010|applescript|  32305|
|2010|     delphi|  13065|
|2011| javascript| 809078|
|2011|       java| 389834|
|2011|        php| 246770|
|2011|          c| 238277|
|2011|objective-c| 218934|
|2011|     python| 203180|
|2011|       bash|  60805|
|2011|       ruby|  39223|
|2011|       perl|  28502|
|2011|     matlab|  18816|
+----+-----------+-------+
only showing top 20 rows



In [76]:
parsed_posts_sample_top10.write.mode('overwrite').parquet("top_10_languages_per_year_between_2010_and_2020.parquet")